# Limpieza — LEGÍTIMAS (Sector: )

**Objetivo**: normalizar, deduplicar y exportar URLs legítimas para su uso en el baseline.

- **Entradas**: `raw/legitimas/<sector>/*.csv`
- **Salida**: `processed/legitimas/<sector>/legitimas_<sector>_limpio.csv`
- **Última actualización**: 2025-08-18
- **Autor**: Alexis Zapico

**Definición de Hecho (DoD)**
1) Carga CSV crudo del sector desde `raw/legitimas/<sector/`.
2) Filtra nulos y URLs inválidas.
3) Deduplica por `url`.
4) Calcula métricas básicas de validación.
5) Exporta a `processed/legitimas/<sector/legitimas_<sector>_limpio.csv`.
6) Escribe log en `docs/daily_log.md`.


In [1]:
# ==== LIBRERÍAS ====
import os, re, glob                        # librerías estándar: sistema de archivos, regex y búsqueda de archivos
import pandas as pd                        # pandas: análisis y manipulación de datos
from datetime import datetime              # para poner fecha y hora en logs
from urllib.parse import urlsplit, urlunsplit, unquote  # utilidades para parsear y reconstruir URLs
from pathlib import Path                   # manejo de rutas multiplataforma
import validators                          # librería externa para validar URLs sintácticamente


In [2]:
def find_repo_root(start: Path = Path().resolve()):
    """Sube hasta 10 niveles buscando un directorio que parezca raíz de repo (data/.git/README.md)."""
    p = start
    for _ in range(10):
        if (p / "data").exists() or (p / ".git").exists() or (p / "README.md").exists():
            return p
        p = p.parent
    return Path().resolve()

REPO_ROOT = find_repo_root()
DATA_DIR = REPO_ROOT / "data"

SECTOR = "<sector>"  

RAW_DIR = DATA_DIR / "raw" / "legitimas" / SECTOR
PROCESSED_DIR = DATA_DIR / "processed" / "legitimas" / SECTOR
OUT_FILE = PROCESSED_DIR / f"legitimas_{SECTOR}_limpio.csv"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
(REPO_ROOT / "docs").mkdir(exist_ok=True)

print("REPO_ROOT:", REPO_ROOT)
print("RAW_DIR:", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)

REPO_ROOT: /Users/test/Desktop/phishing-detector
RAW_DIR: /Users/test/Desktop/phishing-detector/data/raw/legitimas/banca
PROCESSED_DIR: /Users/test/Desktop/phishing-detector/data/processed/legitimas/banca


In [3]:
def es_url_valida(u: str) -> bool:
    if not isinstance(u, str): 
        return False
    u = u.strip()
    if not u or not u.startswith(("http://","https://")):
        return False
    try:
        return bool(validators.url(u))
    except Exception:
        return False

def normalizar_url(u: str) -> str:
    if not isinstance(u, str): 
        return ""
    u = unquote(u.strip())
    u = re.sub(r"\s+", "", u)
    try:
        sp = urlsplit(u)
        scheme = (sp.scheme or "").lower()
        netloc = (sp.netloc or "").lower()
        path, query = sp.path or "", sp.query or ""
        return urlunsplit((scheme, netloc, path, query, ""))
    except Exception:
        return u

In [4]:
# === CARGA CRUDA ROBUSTA (banca) ===
CANDIDATES = ["url", "URL", "Url", "enlace", "link", "href"]

# Busca .csv y .CSV
files = sorted(list(RAW_DIR.glob("*.csv")) + list(RAW_DIR.glob("*.CSV")))
print("Buscando en:", RAW_DIR.resolve())
print("Encontrados:", [p.name for p in files])
assert files, f"No hay CSV en {RAW_DIR}. Revisa la ruta y los nombres."

def read_csv_tolerant(path: Path):
    for args in (
        dict(),  # por defecto
        dict(encoding="utf-8", engine="python", on_bad_lines="skip"),
        dict(encoding="latin-1", engine="python", on_bad_lines="skip"),
    ):
        try:
            return pd.read_csv(path, **args)
        except Exception:
            continue
    raise RuntimeError(f"No se pudo leer {path.name} con los encodings probados.")

dfs, per_file_counts, skipped = [], [], []
for f in files:
    try:
        d = read_csv_tolerant(f)
        col_url = next((c for c in CANDIDATES if c in d.columns), None)
        if not col_url:
            raise KeyError(f"{f.name} sin columna de URL reconocida {CANDIDATES}")
        d = d[[col_url]].rename(columns={col_url: "url"}).copy()
        d["__source_file"] = f.name
        dfs.append(d)
        per_file_counts.append((f.name, len(d)))
    except Exception as e:
        print(f"[WARN] Saltando {f.name}: {e}")
        skipped.append((f.name, str(e)))

assert dfs, "Ningún CSV válido. Revisa los [WARN] y corrige."
df_raw = pd.concat(dfs, ignore_index=True)

print("Archivos leídos:")
for n, k in per_file_counts:
    print(f" - {n}: {k} filas")
print("TOTAL filas crudas:", len(df_raw))
df_raw.head(3)


Buscando en: /Users/test/Desktop/phishing-detector/data/raw/legitimas/banca
Encontrados: ['banca_legitimas.csv', 'banca_legitimas_2.csv', 'legitimas_banca.csv']
[WARN] Saltando legitimas_banca.csv: "legitimas_banca.csv sin columna de URL reconocida ['url', 'URL', 'Url', 'enlace', 'link', 'href']"
Archivos leídos:
 - banca_legitimas.csv: 373 filas
 - banca_legitimas_2.csv: 756 filas
TOTAL filas crudas: 1129


,url,__source_file
0,https://www.caixabank.es/particular/home/parti...,banca_legitimas.csv
1,https://www.caixabank.es/particular/general/co...,banca_legitimas.csv
2,https://www.caixabank.es/particular/general/co...,banca_legitimas.csv


In [5]:
df = df_raw.copy()
df["url"] = df["url"].map(normalizar_url)
df = df[df["url"].map(es_url_valida)]
df = df.dropna(subset=["url"])
df = df.drop_duplicates(subset=["url"]).reset_index(drop=True)
print("Filas tras limpieza:", len(df))
df.head(5)


Filas tras limpieza: 1111


,url,__source_file
0,https://www.caixabank.es/particular/home/parti...,banca_legitimas.csv
1,https://www.caixabank.es/particular/general/co...,banca_legitimas.csv
2,https://uniblog.unicajabanco.es,banca_legitimas.csv
3,https://www.unicajabanco.es/es/particulares/cu...,banca_legitimas.csv
4,https://www.unicajabanco.es/es/particulares/cu...,banca_legitimas.csv


In [6]:
# Métricas
resumen = {
    "filas_crudas": len(df_raw),
    "filas_limpias": len(df),
    "%https": round(df["url"].str.startswith("https://").mean()*100, 2),
    "longitud_media": round(df["url"].str.len().mean(), 2),
}
print(resumen)

# Export
df.to_csv(OUT_FILE, index=False)
print("Guardado en:", OUT_FILE)

# Log
log_line = (
    f"{datetime.now():%Y-%m-%d} | limpieza_legitimas_{SECTOR} | "
    f"crudo={len(df_raw)} | limpio={len(df)} | out={OUT_FILE}\n"
)
with open(REPO_ROOT / "docs" / "daily_log.md", "a") as f:
    f.write(log_line)
print("Log añadido:", log_line.strip())


{'filas_crudas': 1129, 'filas_limpias': 1111, '%https': np.float64(99.46), 'longitud_media': np.float64(91.48)}
Guardado en: /Users/test/Desktop/phishing-detector/data/processed/legitimas/banca/legitimas_banca_limpio.csv
Log añadido: 2025-08-18 | limpieza_legitimas_banca | crudo=1129 | limpio=1111 | out=/Users/test/Desktop/phishing-detector/data/processed/legitimas/banca/legitimas_banca_limpio.csv
